# Alpaca-py crypto trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/crypto-trading-basic.ipynb)

In [29]:
from google.colab import userdata


In [30]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/

api_key = userdata.get('AlpacaAPIKey')
secret_key = userdata.get('AlpacaSecretKey')

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

SecretNotFoundError: Secret AlpacaAPIKey does not exist.

- This notebook shows how to use alpaca-py with crypto trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for crypto as an example.

In [31]:
# install alpaca-py
! python3 -m pip install alpaca-py

In [11]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.crypto import CryptoHistoricalDataClient
from alpaca.trading.stream import TradingStream
from alpaca.data.live.crypto import CryptoDataStream

from alpaca.data.requests import (
    CryptoBarsRequest,
    CryptoQuoteRequest,
    CryptoTradesRequest,
    CryptoLatestQuoteRequest
    )
from alpaca.trading.requests import (
    GetAssetsRequest,
    MarketOrderRequest,
    LimitOrderRequest,
    StopLimitOrderRequest,
    GetOrdersRequest,
    ClosePositionRequest
)
from alpaca.trading.enums import (
    AssetClass,
    AssetStatus,
    OrderSide,
    OrderType,
    TimeInForce,
    QueryOrderStatus
)
from alpaca.common.exceptions import APIError

In [ ]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# check version of alpaca-py
alpaca.__version__

# Trading Client

In [32]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [12]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA32AZSRCSBZ',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2024, 12, 29, 14, 13, 59, 4140, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('a2c721f7-2731-4876-b895-dc0b22e88d1a'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'options_approved_level': 2,
    'options_buying_power': '100000',
    'options_trading_level': 2,
    'pattern_day_trader': False,
    'pending_transfer_in': None,
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': 

In [13]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

{   'dtbp_check': <DTBPCheck.ENTRY: 'entry'>,
    'fractional_trading': True,
    'max_margin_multiplier': '4',
    'max_options_trading_level': None,
    'no_shorting': False,
    'pdt_check': <PDTCheck.ENTRY: 'entry'>,
    'ptp_no_exception_entry': False,
    'suspend_trade': False,
    'trade_confirm_email': <TradeConfirmationEmail.ALL: 'all'>}

In [14]:
# get list of crypto pairs
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  asset_class=AssetClass.CRYPTO,
  status=AssetStatus.ACTIVE
)
assets = trade_client.get_all_assets(req)
assets[:2]

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('38f8a0a4-399c-4f5a-8c0d-7435faef70d1'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 1.00086074,
     'min_trade_increment': 1e-09,
     'name': 'USDC/USD pair',
     'price_increment': 0.0001,
     'shortable': False,
     'status': <AssetStatus.ACTIVE: 'active'>,
     'symbol': 'USDC/USD',
     'tradable': True},
 {   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'attributes': [],
     'easy_to_borrow': False,
     'exchange': <AssetExchange.CRYPTO: 'CRYPTO'>,
     'fractionable': True,
     'id': UUID('a1733398-6acc-4e92-af24-0d0667f78713'),
     'maintenance_margin_requirement': 100.0,
     'marginable': False,
     'min_order_size': 0.000296709,
     'min_trade_increment': 1e-09,
     'name': 'Ethereum / US Dollar',
     'price_inc

### Orders

In [15]:
# we will place orders which Alapca trading platform supports
# - order types for crypto: market, limit, stop_limit
# - time_in_force for crypto: gtc, ioc.
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder


# we will place orders for symbol: BTC/USD in this example
symbol = "BTC/USD"

In [16]:
# simple, market order
# you can specify:
# or notional value (e.g. 100 USD) (which is in the next example)
req = MarketOrderRequest(
    symbol = symbol,
    qty = 0.01,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '103cbeac-baff-481b-b658-e37b7009060f',
    'created_at': datetime.datetime(2024, 12, 29, 14, 37, 14, 382033, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('9edc0e27-5b60-45a0-99e7-ba17f8ef8243'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12

In [17]:
# simple, market order, notional

req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '345d363a-b2a5-4a9a-a129-5c103f44e451',
    'created_at': datetime.datetime(2024, 12, 29, 14, 38, 13, 952589, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('4951a60c-195a-4ed6-a62a-eaa9c36f9e3e'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 12

In [18]:
# simple, limit order
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 60000,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': 'ab63f7c5-2d31-449e-80be-25d0e3c0d2df',
    'created_at': datetime.datetime(2024, 12, 29, 14, 38, 34, 832063, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('4852b174-6464-4c2e-be6b-00e04efbf9e4'),
    'legs': None,
    'limit_price': '60000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 1

In [19]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 0.01,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 55000,
                    stop_price = 60000
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': 'c6242abb-5138-453c-b619-b8440bd97453',
    'created_at': datetime.datetime(2024, 12, 29, 14, 38, 43, 393478, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('b9f94249-eb35-48eb-9dae-359eaf81a7d0'),
    'legs': None,
    'limit_price': '55000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.NEW: 'new'>,
    'stop_price': '60000',
    'submitted_at': datetime.datetime(2024, 12, 

In [20]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': 'c6242abb-5138-453c-b619-b8440bd97453',
     'created_at': datetime.datetime(2024, 12, 29, 14, 38, 43, 393478, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('b9f94249-eb35-48eb-9dae-359eaf81a7d0'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
     'submitted_at': d

In [21]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': 'c6242abb-5138-453c-b619-b8440bd97453',
     'created_at': datetime.datetime(2024, 12, 29, 14, 38, 43, 393478, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('b9f94249-eb35-48eb-9dae-359eaf81a7d0'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'position_intent': <PositionIntent.BUY_TO_OPEN: 'buy_to_open'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': '60000',
     'submitted_at': d

In [22]:
# cancel all open orders
trade_client.cancel_orders()

[{   'body': None,
     'id': UUID('4852b174-6464-4c2e-be6b-00e04efbf9e4'),
     'status': 200},
 {   'body': None,
     'id': UUID('b9f94249-eb35-48eb-9dae-359eaf81a7d0'),
     'status': 200}]

### Positions

In [ ]:
# '/' not being supported for position calls
symbol="BTCUSD"

In [ ]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

In [ ]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


In [ ]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

In [ ]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [23]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

keyboard interrupt, bye


# Market Data (Historical)

In [33]:
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient()

symbols = ['BTC/USD']

In [36]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/cryptobars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoBarsRequest(
    symbol_or_symbols = symbols,
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_bars(req).df

,,open,high,low,close,volume,trade_count,vwap
symbol,timestamp,,,,,,,
BTC/USD,2024-12-29 15:00:00+00:00,94510.4,94604.0905,94263.285,94604.0905,0.2,1.0,94532.7


In [28]:
# get historical quote by symbol
# ref. https://docs.alpaca.markets/reference/cryptoquotes-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoQuoteRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 10,                                               # specify limit
)
crypto_historical_data_client.get_crypto_quotes(req).df

bid_price  bid_size  ask_price  \
symbol  timestamp                                                          
BTC/USD 2024-12-28 14:43:48.645690+00:00  94386.900    1.6220  94537.940   
        2024-12-28 14:43:48.645798+00:00  94478.300    0.7975  94537.940   
        2024-12-28 14:43:48.645881+00:00  94478.300    0.7975  94574.703   
        2024-12-28 14:43:48.646128+00:00  94478.300    0.7975  94582.020   
        2024-12-28 14:44:47.847019+00:00  94422.651    1.5980  94582.020   
        2024-12-28 14:44:47.847184+00:00  94422.651    1.5980  94645.330   
        2024-12-28 14:44:47.847370+00:00  94422.651    1.5980  94529.200   
        2024-12-28 14:44:47.847559+00:00  94394.310    0.8091  94529.200   
        2024-12-28 14:45:07.518654+00:00  94394.310    0.8091  94571.700   
        2024-12-28 14:45:07.518669+00:00  94376.100    1.5920  94571.700   

                                          ask_size  
symbol  timestamp                                   
BTC/USD 2024-12-28 14:43:48.645690+00:00   0.79698  
        2024-12-28 14:43:48.645798+00:00   0.79698  
        2024-12-28 14:43:48.645881+00:00   1.59900  
        2024-12-28 14:43:48.646128+00:00   0.80900  
        2024-12-28 14:44:47.847019+00:00   0.80900  
        2024-12-28 14:44:47.847184+00:00   1.60431  
        2024-12-28 14:44:47.847370+00:00   0.80320  
        2024-12-28 14:44:47.847559+00:00   0.80320  
        2024-12-28 14:45:07.518654+00:00   1.59229  
        2024-12-28 14:45:07.518669+00:00   1.59229

In [29]:
# get historical trades by symbol
req = CryptoTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 1),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
crypto_historical_data_client.get_crypto_trades(req).df

price      size  \
symbol  timestamp                                             
BTC/USD 2024-12-28 14:45:07.953147+00:00  94612.6  0.000720   
        2024-12-28 14:48:33.906680+00:00  94316.3  0.000515   

                                                           id  
symbol  timestamp                                              
BTC/USD 2024-12-28 14:45:07.953147+00:00  8608481685453229903  
        2024-12-28 14:48:33.906680+00:00  4017472475364347425

In [30]:
# get latest quotes by symbol
req = CryptoLatestQuoteRequest(
    symbol_or_symbols = [symbol],
)
res = crypto_historical_data_client.get_crypto_latest_quote(req)
res

{'BTC/USD': {   'ask_exchange': None,
     'ask_price': 94190.11,
     'ask_size': 0.81071,
     'bid_exchange': None,
     'bid_price': 94031.657,
     'bid_size': 1.61309,
     'conditions': None,
     'symbol': 'BTC/USD',
     'tape': None,
     'timestamp': datetime.datetime(2024, 12, 29, 14, 42, 25, 824521, tzinfo=TzInfo(UTC))}}

In [40]:
import requests

url = "https://data.alpaca.markets/v1beta3/crypto/us/snapshots?symbols=BTC%2FUSD"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"message":"Invalid location: loc"}



# New Section

In [53]:
import yfinance as yf
from datetime import datetime, timedelta

In [68]:
prices = yf.download(["AAPL"], interval='5m')["Adj Close"]

def rsi(prices, n=14):
    """Compute the RSI given prices

    :param prices: pandas.Series
    :return: rsi
    """

    # Calculate the difference between the current and previous close price
    delta = prices.diff()

    # Calculate the sum of all positive changes
    gain = delta.where(delta > 0, 0)

    # Calculate the sum of all negative changes
    loss = -delta.where(delta < 0, 0)

    # Calculate the average gain over the last n periods
    avg_gain = gain.rolling(n).mean()

    # Calculate the average loss over the last n periods
    avg_loss = loss.rolling(n).mean()

    # Calculate the relative strength
    rs = avg_gain / avg_loss

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

rsi_current = rsi(prices)

[*********************100%***********************]  1 of 1 completed


In [69]:
rsi_current[rsi_current.index==rsi_current.index.max()]

Ticker,AAPL
Datetime,
2024-12-27 20:55:00+00:00,60.992243
